In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
from datasets import load_dataset

ds = load_dataset("sehyun66/Finnhub-News", "clean")

In [ ]:
import pandas as pd

pdds = pd.DataFrame(ds['clean'])
pdds = pdds.drop(columns=['image','source','url','id'])
pdds

In [ ]:
pdds['datetime'] =pd.to_datetime(pdds['datetime'], unit='ms')
pdds

In [ ]:
aapl = pdds[pdds['related'].str.contains('|'.join(['AAPL', 'GOOGL','AMZN']))]
#aapl.dropna(axis=0, how='any')
aapl = aapl.drop(aapl[aapl['summary']==''].index)
aapl = aapl.drop(aapl[aapl['headline']==''].index)
aapl



In [ ]:
import numpy as np

headlines_array = np.array(aapl)
i=0
rango = 100
inputsArray =[]

while len(headlines_array) > i*rango:
    headlines_list = list(headlines_array[i*rango:(i+1)*rango, 4])
    inputsArray.append(tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt'))
    i = i + 1

In [ ]:
import torch

positive = []
negative = []
neutral = []

for inputs in inputsArray:
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    positive.extend(predictions[:, 0].tolist())
    negative.extend(predictions[:, 1].tolist())
    neutral.extend(predictions[:, 2].tolist())


In [ ]:
model.config.id2label

table = {'Date':headlines_array[:,0],
         'Headline':headlines_array[:,4],
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Date","Headline", "Positive", "Negative", "Neutral"])
df = df.sort_values("Date")
df
